In [205]:
# manipulando tabela do brasileirão série A a partir de 2012
# para verificação de estratégia de Poisson - ainda falta terminar

import csv 
import statistics as st
import math
import numpy as np

# Distribuição de Poisson para resultados de futebol
def poisson(time1,time2,mgf_ccasa,mgs_ccasa,mgf_cfora,mgs_cfora,rodada):
    mgf_tcasa = time1['mgf_tcasa']
    mgs_tcasa = time1['mgs_tcasa']
    mgf_tfora = time2['mgf_tfora']
    mgs_tfora = time2['mgf_tfora']
    
    # tratamento de zeros
    if mgf_tcasa == 0:
        mgf_tcasa = 0.1
        
    if mgs_tcasa == 0:
        mgs_tcasa = 0.1
        
    if mgf_tfora == 0:
        mgf_tfora = 0.1
        
    if mgs_tfora == 0:
        mgs_tfora = 0.1
        
    if mgf_ccasa == 0:
        mgf_ccasa = 0.1
        
    if mgs_ccasa == 0:
        mgs_ccasa = 0.1
        
    if mgf_cfora == 0:
        mgf_cfora = 0.1
        
    if mgs_cfora == 0:
        mgs_cfora = 0.1
    
    media_getc = mgf_tcasa / mgf_ccasa * mgs_tfora / mgs_cfora * mgf_ccasa
    media_getf = mgf_tfora / mgf_cfora * mgs_tcasa / mgs_ccasa * mgf_cfora

    # p/ prob. do número de gols
    probf_casa = [0,0,0,0,0,0,0,0,0,0,0]
    probf_fora = [0,0,0,0,0,0,0,0,0,0,0]
    probs_casa = [0,0,0,0,0,0,0,0,0,0,0]
    probs_fora = [0,0,0,0,0,0,0,0,0,0,0]

    # p/ prob. de vitória, empate e derrota
    probv_casa = 0
    probv_fora = 0
    prob_empate = 0

    # p/ prob. dupla chance
    dupla_casa = 0
    dupla_fora = 0

    # p/ prob. empate anula
    emp_ac = 0
    emp_af = 0

    # p/ prob. under e over
    under_05 = 0
    over_05 = 0
    under_15 = 0
    over_15 = 0
    under_25 = 0
    over_25 = 0
    under_35 = 0
    over_35 = 0

    # p/ prob. ambas marcam
    ambas_sim = 0
    ambas_nao = 0

    prob_placar = np.zeros((11,11), dtype=np.float64)
    placar_ordenado = []
    pos_placar_ordenado = []
    posicao = 0
    cont = 0

    # prob. de gols do time da casa e fora
    for i in range(11):
        probf_casa[i] = (math.exp(-media_getc)*media_getc**i)/math.factorial(i)
        probf_fora[i] = (math.exp(-media_getf)*media_getf**i)/math.factorial(i)

    # prob. de placares do jogo; prob. de 1x2; prob. de under e over
    # i -> num de gols do time da casa; j -> num de gols do time de fora
    for i in range(11):
        for j in range(11):
            # prob. de 1x2
            if i > j:
                probv_casa += probf_casa[i] * probf_fora[j] * 100
            elif j > i:
                probv_fora += probf_casa[i] * probf_fora[j] * 100
            else: 
                prob_empate += probf_casa[i] * probf_fora[j] * 100

            # prob. de under e over
            # under/over 0.5
            if i + j <= 0:
                under_05 += probf_casa[i] * probf_fora[j] * 100

            # under/over 1.5
            if i + j <= 1:
                under_15 += probf_casa[i] * probf_fora[j] * 100

            # under/over 2.5
            if i + j <= 2:
                under_25 += probf_casa[i] * probf_fora[j] * 100

            # under/over 3.5
            if i + j <= 3:
                under_35 += probf_casa[i] * probf_fora[j] * 100

            # prob. ambas marcam
            if (i > 0) and (j > 0):
                ambas_sim += probf_casa[i] * probf_fora[j] * 100

            # prob. de placar exato
            prob_placar[i][j] = probf_casa[i] * probf_fora[j] * 100
            placar_ordenado.append(prob_placar[i][j])
            pos_placar_ordenado.append(placar_ordenado[cont])
            cont += 1
            # print(f'{i} x {j} -> {prob_placar[i][j]:.2f}%')

    # prob. dupla chance
    dupla_casa = (probv_casa + prob_empate) 
    dupla_fora = (probv_fora + prob_empate)

    # prob. empate anula
    emp_ac = probv_casa/(probv_casa + probv_fora) * 100
    emp_af = 100 - emp_ac

    # prob. over 0.5
    over_05 = 100 - under_05

    # prob. over 1.5
    over_15 = 100 - under_15

    # prob. over 2.5
    over_25 = 100 - under_25

    # prob. over 3.5
    over_35 = 100 - under_35

    # prob. ambas marcam não
    ambas_nao = 100 - ambas_sim

    placar_ordenado.sort(reverse = True)
    
    print(f'Rodada {rodada}')
    print(f'prob. de vitória casa => {probv_casa:.2f}%')
    print(f'prob. de vitória visitante => {probv_fora:.2f}%')
    print(f'prob. de empate: {prob_empate:.2f}%\n')


    '''print(f'\nPlacar Ordenado\n')
    # for i in range(121): 121 é o máximo. Nesse range pode-se escolher quantos resultados corretos exibir
    for i in range(10):
        posicao = pos_placar_ordenado.index(placar_ordenado[i])

        # Resultados Corretos
        print(f'{int(posicao/11)} x {posicao%11} -> {placar_ordenado[i]:.2f}%')

    # 1x2
    print(f'\nprob. de vitória casa => {probv_casa:.2f}%')
    print(f'prob. de vitória visitante => {probv_fora:.2f}%')
    print(f'prob. de empate: {prob_empate:.2f}%')

    # dupla chance
    print(f'\nprob. dupla chance casa => {dupla_casa:.2f}%')
    print(f'prob. dupla chance visitante => {dupla_fora:.2f}%')

    # empate anula
    print(f'\nprob. empate anula casa => {emp_ac:.2f}%')
    print(f'prob. empate anula visitante: => {emp_af:.2f}%')

    # under/over 0.5
    print(f'\nprob. under 0.5 => {under_05:.2f}%')
    print(f'prob. over 0.5 => {over_05:.2f}%')

    # under/over 1.5
    print(f'\nprob. under 1.5 => {under_15:.2f}%')
    print(f'prob. over 1.5 => {over_15:.2f}%')

    # under/over 2.5
    print(f'\nprob. under 2.5 => {under_25:.2f}%')
    print(f'prob. over 2.5 => {over_25:.2f}%')

    # under/over 3.5
    print(f'\nprob. under 3.5 => {under_35:.2f}%')
    print(f'prob. over 3.5 => {over_35:.2f}%')

    # ambas marcam
    print(f'\nprob. ambas marcam sim => {ambas_sim:.2f}%')
    print(f'prob. ambas marcam não => {ambas_nao:.2f}%')'''
    pass

    
# base de dados
linha_inicial = False
time_casa = ''
time_fora = ''
num_jogos = 0
num_gols = 0
media_gols = 0

mgf_tcasa = 0
mgs_tcasa = 0
mgf_tfora = 0
mgs_tfora = 0
mgf_ccasa = 0
mgs_ccasa = 0
mgf_cfora = 0
mgs_cfora = 0
time = [''] * 20

tgf_ccasa = 0
tgf_cfora = 0

cont1 = 0
cont_jogos = 0
rodada = 1
cont_geral = 1
cont_ind = [0] * 20

time1 = ''
time2 = ''

with open('BRA.csv', 'r', encoding = 'utf-8', newline = '') as al:
    arq1 = csv.DictReader(al)
    
    for linha in arq1:  
        # ano de 2012
        if (linha['Date'].split('/')[2] == '2012'): 
        
            # media de gols dos times e do campeonato
            tgf_ccasa += float(linha['HG'])
            tgf_cfora += float(linha['AG'])
            mgf_ccasa = tgf_ccasa/cont_geral
            mgs_ccasa = tgf_cfora/cont_geral
            mgf_cfora = mgs_ccasa
            mgs_cfora = mgf_ccasa
            
            # carregando os times e o número de jogos de cada
            if cont1 < 10:
                cont_ind[cont1] += 1
                cont_ind[cont1+10] += 1
                time[cont1] = {'time':linha['Home'],'num_jogosc':cont_ind[cont1],'num_jogosf':0,'tgf_tcasa':int(linha['HG']),'tgs_tcasa':int(linha['AG']),'tgf_tfora':0,'tgs_tfora':0,'mgf_tcasa':float(linha['HG'])/cont_ind[cont1],'mgs_tcasa':float(linha['AG'])/cont_ind[cont1],'mgf_tfora':0,'mgs_tfora':0}
                time[cont1+10] = {'time':linha['Away'],'num_jogosc':0,'num_jogosf':cont_ind[cont1+10],'tgf_tcasa':0,'tgs_tcasa':0,'tgf_tfora':int(linha['AG']),'tgs_tfora':int(linha['HG']),'mgf_tcasa':0,'mgs_tcasa':0,'mgf_tfora':float(linha['AG'])/cont_ind[cont1+10],'mgs_tfora':float(linha['HG'])/cont_ind[cont1+10]}
                cont1 += 1

            # contando jogos
            if cont_jogos <= 10:
                cont_jogos += 1

            # atualizando os dados dos times por jogo
            if cont_geral >= 10: 
                for i in range(20):
                    if linha['Home'] == time[i]['time']: 
                        time[i]['num_jogosc'] += 1
                        time[i]['tgf_tcasa'] += int(linha['HG'])
                        time[i]['tgs_tcasa'] += int(linha['AG'])
                       
                        time[i]['mgf_tcasa'] = time[i]['tgf_tcasa']/time[i]['num_jogosc']
                        time[i]['mgs_tcasa'] = time[i]['tgs_tcasa']/time[i]['num_jogosc']
                        
                    if linha['Away'] == time[i]['time']:
                        time[i]['num_jogosf'] += 1
                        time[i]['tgf_tfora'] += int(linha['AG'])
                        time[i]['tgs_tfora'] += int(linha['HG'])

                        time[i]['mgf_tfora'] = time[i]['tgf_tfora']/time[i]['num_jogosf']
                        time[i]['mgs_tfora'] = time[i]['tgs_tfora']/time[i]['num_jogosf']
           
            #contando rodadas - a primeira vez que entra aqui é a rodada 2
            if cont_jogos > 10:
                # atualizando o número de rodadas
                rodada += 1
                cont_jogos = 1

                #calculando dados da rodada
                for i in range(20):  
                    if linha['Home'] == time[i]['time']:
                        time1 = time[i]
                    if linha['Away'] == time[i]['time']:
                        time2 = time[i]
                        
                poisson(time1,time2,mgf_ccasa,mgs_ccasa,mgf_cfora,mgs_cfora,rodada)
                
    cont_geral += 1

Rodada 2
prob. de vitória casa => 0.08%
prob. de vitória visitante => 0.11%
prob. de empate: 99.81%

Rodada 3
prob. de vitória casa => 0.43%
prob. de vitória visitante => 0.52%
prob. de empate: 99.05%

Rodada 4
prob. de vitória casa => 0.02%
prob. de vitória visitante => 0.03%
prob. de empate: 99.94%

Rodada 5
prob. de vitória casa => 0.02%
prob. de vitória visitante => 0.03%
prob. de empate: 99.95%

Rodada 6
prob. de vitória casa => 0.29%
prob. de vitória visitante => 0.49%
prob. de empate: 99.23%

Rodada 7
prob. de vitória casa => 0.23%
prob. de vitória visitante => 0.33%
prob. de empate: 99.44%

Rodada 8
prob. de vitória casa => 0.19%
prob. de vitória visitante => 0.01%
prob. de empate: 99.79%

Rodada 9
prob. de vitória casa => 0.08%
prob. de vitória visitante => 1.20%
prob. de empate: 98.72%

Rodada 10
prob. de vitória casa => 0.01%
prob. de vitória visitante => 0.01%
prob. de empate: 99.98%

Rodada 11
prob. de vitória casa => 0.65%
prob. de vitória visitante => 0.89%
prob. de empa

In [131]:
def t(c1):
    print(c1['novo'])


cont = [0] * 3
c1 = {'conta':cont[0],'novo':cont[0]}
c1['conta'] = 3
c1['novo'] = c1['conta'] + 3
t(c1)

6


In [ ]:
if linha['Home'] == time[i]['time']:
                        time1 = time[i]
                    if linha['Away'] == time[i]['time']:
                        time2 = time[i]